In [ ]:
"""
Created on Fri Oct 27 10:34 2023

Look at temperature data from CMIP

Author: @claraburgard
"""

In [ ]:
import xarray as xr


In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'


Models

In [ ]:
inputpath_tas = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/CMIP_DATA/tas_data/'

In [ ]:
mod = 'ACCESS-ESM1-5' 
scenario = 'ssp245'

if mod in ['CNRM-CM6-1','CNRM-ESM2-1']:
    ens_run = 'r1i1p1f2'
    to2300 = False
elif mod in ['GISS-E2-1-H']:
    ens_run = 'r1i1p1f2'
    to2300 = True
elif mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2-WACCM','CanESM5','IPSL-CM6A-LR','MRI-ESM2-0']:
    ens_run = 'r1i1p1f1'
    to2300 = True
elif mod in ['MPI-ESM1-2-HR','GFDL-CM4','GFDL-ESM4']:
    ens_run = 'r1i1p1f1'
    to2300 = False
elif mod == 'UKESM1-0-LL':
    ens_run = 'r4i1p1f2'
    to2300 = True     
elif mod == 'CESM2':
    ens_run = 'r11i1p1f1'
    to2300 = False        


if scenario == 'historical':
    yystart = 1980 #1850
    yyend = 2014
elif scenario == 'ssp245':
    yystart = 2015
    yyend = 2100  
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

In [ ]:
ens_run = 'r11i1p1f2'
tas_scen_list = []

tas_mod_hist = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_historical_r11i1p1f1_gn_185001-201412_fldmean_ymean.nc')
tas_mod_hist['time'] = tas_mod_hist.time.dt.year
tas_mod_hist_clean = tas_mod_hist['tas'].squeeze()
tas_scen_list.append(tas_mod_hist_clean.assign_coords({'scenario':'historical'}))

for scen in ['ssp126','ssp245','ssp585']:
    tas_mod_scen = xr.open_dataset(inputpath_tas+mod+'/tas_Amon_'+mod+'_'+scen+'_r11i1p1f1_gn_201501-210012_fldmean_ymean.nc')
    tas_mod_scen['time'] = tas_mod_scen.time.dt.year
    tas_scen_list.append(tas_mod_scen['tas'].squeeze().assign_coords({'scenario':scen}))

tas_all_scen = xr.concat(tas_scen_list,dim='scenario')

In [ ]:
for scen in ['historical','ssp126','ssp245','ssp585']:
    tas_all_scen.sel(scenario=scen).plot()


Observations

In [ ]:
inputpath_raw = home_path+'/DATA/SUMMER_PAPER/raw/'
HadCRUT_temp = xr.open_dataset(inputpath_raw + 'HadCRUT.5.0.0.0.analysis.summary_series.global.annual.nc')
HadCRUT_tas = HadCRUT_temp['tas_mean']
HadCRUT_tas['time'] = HadCRUT_temp.time.dt.year

In [ ]:
anom_baseline = HadCRUT_tas.sel(time=range(1850,1880)).mean('time')
anom_picontrol = HadCRUT_tas - anom_baseline

In [ ]:
anom_mod_scen = tas_all_scen - tas_all_scen.sel(scenario='historical').sel(time=range(1850,1880)).mean('time')

In [ ]:
for scen in ['historical','ssp126','ssp245','ssp585']:
    anom_mod_scen.sel(scenario=scen).plot()
    anom_picontrol.plot(color='k')

Masks

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)